## Fenics program Runtime

In [1]:
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from dataloader import *
from functools import reduce
from torch.autograd import Variable
from tqdm import tqdm
%matplotlib inline

In [2]:
class Model(nn.Module):
    def __init__(self,num_layers):
        super(Model,self).__init__()
        self.model1 = nn.ModuleList([nn.Sequential(nn.ConstantPad2d(1,0), nn.Conv2d(1,1,3,bias=False)) for i in range(num_layers)])
        self.model2 = nn.ModuleList([nn.Sequential(nn.ConstantPad2d(1,0), nn.Conv2d(1,1,3,bias=False)) for i in range(num_layers)])
        self.model3 = nn.ModuleList([nn.Sequential(nn.ConstantPad2d(1,0), nn.Conv2d(1,1,3,bias=False)) for i in range(num_layers)])
        self.model4 = nn.ModuleList([nn.Sequential(nn.ConstantPad2d(1,0), nn.Conv2d(1,1,3,bias=False)) for i in range(num_layers)])
#         self.model = nn.Sequential(self.model1)
    def forward(self,x):
#         print(x)
        x1 = reduce(lambda acc, el: el(acc), self.model1, x)
        x2 = reduce(lambda acc, el: el(acc), self.model2, x)
        x3 = reduce(lambda acc, el: el(acc), self.model3, x)
        x4 = reduce(lambda acc, el: el(acc), self.model4, x)
        
        x = torch.cat((x1,x2,x3,x4),1)
        return x

In [3]:
model = Model(3)
# delta X
nx = 65.
delx = 2.*3.14/nx
delt = 0.2
epsilon = 0.50
d_ = torch.from_numpy(np.array([0.0/delx, 0.0/delx, -2./(delx**2), -2./(delx**2)])).type(torch.FloatTensor)
dx = torch.from_numpy(np.array([1.0/delx, 1.0/delx, 1.0/(delx**2), 1.0/(delx**2)])).type(torch.FloatTensor)

In [4]:
def psi_input(u1, u2, epsilon):
    filter_1 = -np.expand_dims(np.expand_dims(np.array([[0, 0, 0],[-0.5, 0, 0.5], [0, 0, 0]]), 0), 0)
    filter_2 = -np.expand_dims(np.expand_dims(np.array([[0, -0.5, 0],[0, 0, 0],[0, 0.5, 0]]), 0), 0) 
    filter_3 = np.expand_dims(np.expand_dims(np.array([[0, 0, 0],[1., 0, 1.],[0, 0, 0]]), 0), 0) #d_2 = -2.
    filter_4 = np.expand_dims(np.expand_dims(np.array([[0, 1., 0],[0, 0, 0],[0, 1., 0]]), 0), 0) #d_2 = -2.
    filter_5 = np.expand_dims(np.expand_dims(np.array([[0, 0, 0],[1., -2., 1.],[0, 0, 0]]), 0), 0) #d_2 = -2.
    filter_6 = np.expand_dims(np.expand_dims(np.array([[0, 1., 0],[0, -2., 0],[0, 1., 0]]), 0), 0) #d_2 = -2.
    Filter_var_d = np.concatenate([filter_1, filter_2, filter_3, filter_4],axis=0)
    Filter_var = np.concatenate([filter_1, filter_2, filter_5, filter_6],axis=0)
    if(torch.cuda.is_available()!=False):
        Filter_var = Variable(torch.FloatTensor(Filter_var).cuda(), requires_grad=False)
        Filter_var_d = Variable(torch.FloatTensor(Filter_var_d).cuda(), requires_grad=False)
    else:
        Filter_var = Variable(torch.FloatTensor(Filter_var), requires_grad=False)
        Filter_var_d = Variable(torch.FloatTensor(Filter_var_d), requires_grad=False)
    u1 = F.pad(u1, (1,1,1,1), mode='constant')
    u2 = F.pad(u2, (1,1,1,1), mode='constant')
    out1 = F.conv2d(u1, Filter_var_d, stride=1, padding=0)
    out2 = F.conv2d(u2, Filter_var, stride=1, padding=0)
    return out1*epsilon+(1-epsilon)*out2

In [5]:
def getG(psi):
    psi.data[:,:,0,:] = 1e-10
    psi.data[:,:,:,0] = 1e-10
    psi.data[:,:,-1,:] = 1e-10
    psi.data[:,:,:,-1] = 1e-10
    return psi

In [6]:
def parameter_mult(psi, parameters):
    out = torch.mul(psi, parameters.unsqueeze(-1).unsqueeze(-1))
    output = torch.sum(out, 1, keepdim=True)
    return output

In [7]:
def phi_input(phi, u, parameter, delta_t, epsilon):
    
    div = 1.0/(1.0 - torch.sum(delt*epsilon*torch.mul(parameter, d_), 1)).unsqueeze(1)
    parameter = torch.mul(parameter, dx)
    lamda = torch.mul(parameter, div)

    psi = phi

    psi = parameter_mult(psi_input(psi, u, epsilon), lamda)*delta_t+parameter_mult(u, div)
    psi = getG(psi)
    w = psi - phi
    # Boundary correction should be here on psi
    #     print(model(input_to_net).shape)

    w = getG(parameter_mult(model(w),  lamda*delta_t*epsilon))
        
    phi = w + psi
#         psi = phi
    return phi

In [20]:
def neural_solver(u, parameter, delta_t, epsilon):
    phi = u
#     iter_ = np.random.randint(1, 20)
    iter_= 15
    for _ in range(iter_):
        phi = phi_input(phi, u, parameter, delta_t, epsilon)
    return phi

In [9]:
def implicit_FDM(u, parameter, delta_t, epsilon):
    div = 1.0/(1.0 - torch.sum(delt*epsilon*torch.mul(parameter, d_), 1)).unsqueeze(1)
    parameter = torch.mul(parameter, dx)
    lamda = torch.mul(parameter, div)
    psi = u
    for i in range(25):
        psi = parameter_mult(psi_input(psi, u, epsilon), lamda)*delta_t+parameter_mult(u, div)
        psi = getG(psi)
        # Boundary correction should be here on psi
    #     print(model(input_to_net).shape)

    return psi

In [10]:
model = torch.load('Model/model_0.5_final.pt',map_location='cpu')
model

Model(
  (model1): ModuleList(
    (0): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (2): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
  )
  (model2): ModuleList(
    (0): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (1): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
    (2): Sequential(
      (0): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
      (1): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), bias=False)
    )
  )
  (model3): M

In [11]:
import time

In [12]:
list_of_parameters = glob.glob('./data/test_data/*.csv',recursive=True)

In [13]:
test_data_ = []
for i in list_of_parameters:
    test_data_.append(os.path.split(i)[1][:-15])

In [14]:
def parameter(filename):
    data = pd.read_csv(filename,header=None)
    output = np.array(data)
    output = torch.tensor(output)
    return output

In [15]:
m = 1
for number in test_data_[m:m+1]:
    data_list = glob.glob('./data/test_data/'+number+'/*.csv')
    data_list = sorted(data_list, key=lambda a: int(os.path.split(a)[1][:-4].zfill(3)))
    parafile = './data/test_data/'+number+'_parameters.csv'

In [21]:
file = open('time_neural15.txt','w+')

In [22]:
start = time.time()
n = len(data_list)
print(n)
delt = 0.2
epsilon = 0.5
for idx in range(1):
    param = parameter(parafile).type(torch.FloatTensor).squeeze().unsqueeze(0)
    data = np.expand_dims(np.expand_dims(parameter(data_list[idx]),axis=0),axis=0)
    data = torch.tensor(data)
    data = data.type(torch.FloatTensor)
    output2 = neural_solver(data, param, delt, epsilon)
file.write(str(time.time() - start))
print(time.time() - start)
file.close()

50
0.10417604446411133


In [ ]:
fenics_time = pd.read_csv('time_fenics.txt')
print(fenics_time)
neural_time = pd.read_csv('time_neural.txt')
print(neural_time)

In [ ]:
ratio = int(98.79823708534241/2.5830504894)
ratio

In [19]:
file = open('time_neural30.txt','w+')
start = time.time()
n = len(data_list)
print(n)
delt = 0.2
epsilon = 0.5
for idx in range(1):
    param = parameter(parafile).type(torch.FloatTensor).squeeze().unsqueeze(0)
    data = np.expand_dims(np.expand_dims(parameter(data_list[idx]),axis=0),axis=0)
    data = torch.tensor(data)
    data = data.type(torch.FloatTensor)
    output2 = neural_solver(data, param, delt, epsilon)
file.write(str(time.time() - start))
print(time.time() - start)
file.close()

50
0.19795775413513184
